
# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prashgup/AI_Beginning/blob/main/Week_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [ ]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__, "| tiktoken", tiktoken.__version__)

torch 2.8.0+cu126 | transformers 4.56.2 | tiktoken 0.11.0


# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [28]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
tokens = []
for sentence in corpus:
  words = sentence.split() # default whitespace tokenizer
  tokens.extend(words)     # add words to the list

# find unique and sorted words
vocab = [PAD, UNK] + sorted(set(tokens))

word2id = {word: idx for idx, word in enumerate(vocab)}
id2word = {idx: word for idx, word in enumerate(vocab)}

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])
print(word2id)
print(id2word)

# 3. Encode / decode
def encode(text):
    tokens = text.split()
    tokenids = [ word2id.get(token, UNK) for token in tokens]
    return tokenids

def decode(ids):
    ## Approach 1
    # tokens = [ id2word.get(tokenid, UNK) for tokenid in ids]
    # return tokens

    # Approach 2
    tokens = []
    for tokenid in ids:
      tokens.append(id2word.get(tokenid, UNK))
    return tokens

    ## Approach 3
    # return [ id2word.get(tokenid, UNK) for tokenid in ids]


# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)

Vocabulary size: 22 words
First 15 vocab entries: ['[PAD]', '[UNK]', 'Large', 'The', 'Tokenization', 'brown', 'converts', 'dog', 'fox', 'jumps', 'language', 'lazy', 'models', 'next', 'numbers']
{'[PAD]': 0, '[UNK]': 1, 'Large': 2, 'The': 3, 'Tokenization': 4, 'brown': 5, 'converts': 6, 'dog': 7, 'fox': 8, 'jumps': 9, 'language': 10, 'lazy': 11, 'models': 12, 'next': 13, 'numbers': 14, 'over': 15, 'predict': 16, 'quick': 17, 'text': 18, 'the': 19, 'to': 20, 'token': 21}
{0: '[PAD]', 1: '[UNK]', 2: 'Large', 3: 'The', 4: 'Tokenization', 5: 'brown', 6: 'converts', 7: 'dog', 8: 'fox', 9: 'jumps', 10: 'language', 11: 'lazy', 12: 'models', 13: 'next', 14: 'numbers', 15: 'over', 16: 'predict', 17: 'quick', 18: 'text', 19: 'the', 20: 'to', 21: 'token'}

Input text : The brown unicorn jumps
Token IDs  : [3, 5, '[UNK]', 9]
Decoded    : ['The', 'brown', '[UNK]', 'jumps']


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [29]:
# 1. Build a fixed vocabulary # a–z + A–Z + padding + unkwown
import string

vocab = list(string.ascii_lowercase + string.ascii_uppercase + '_' + "#")
char2id = {c:id for id, c in enumerate(vocab)}
id2char = {id:c for id, c in enumerate(vocab)}

print(vocab)
print(char2id)
print(id2char)

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    return [ char2id.get(c, '_') for c in text]

def decode(ids):
    return [ id2char.get(id, "_") for id in ids]

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '#']
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '_': 52, '#': 53}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: 'A', 27: 'B', 28: 'C', 

### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [30]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer

# tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast="true")

# 2. Encode / decode
def encode(text):
    return tokenizer.encode(text)

def decode(ids):
    return tokenizer.decode(ids)

def ids2token(ids):
    return tokenizer.convert_ids_to_tokens(ids)

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)
ids2tokens = ids2token(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("IDs to Token :", ids2tokens)
print("Decoded    :", recovered)



Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
IDs to Token : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [31]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken

# tiktokenizer
#tiktokenizer = tiktoken.get_encoding("o200k_base")
#tiktokenizer = tiktoken.get_encoding("cl100k_base")

# tokenizer corresponding to specific model
tiktokenizer = tiktoken.encoding_for_model("gpt-4o")


sentence = "The 🌟 star-player scored 40 points!"
ids = tiktokenizer.encode(sentence)
print("Encoded ids", ids)

tokens = tiktokenizer.decode(ids)
print("Decoded tokens", tokens)

Encoded ids [976, 130321, 253, 8253, 92135, 27207, 220, 1723, 5571, 0]
Decoded tokens The 🌟 star-player scored 40 points!


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [ ]:
import torch.nn as nn
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        """
        YOUR CODE HERE
        """

    def forward(self, x):
        """
        YOUR CODE HERE
        """

In [32]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[ 0.5362, -0.2396,  0.5230],
        [-0.2970,  0.1148,  0.5697]], requires_grad=True)
Bias   : Parameter containing:
tensor([ 0.0543, -0.1495], requires_grad=True)
Output : tensor([ 1.0916, -0.2765], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [33]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
model_name = "gpt2"   # this is the 124M model
gpt2 = GPT2LMHeadModel.from_pretrained(model_name)

num_layers = len(gpt2.transformer.h)
print(f"\nNumber of transformer layers: {num_layers}")

# Inspect first layer details
print("\n=== First transformer block ===")
print(gpt2.transformer.h[0])


Number of transformer layers: 12

=== First transformer block ===
GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [34]:
# Run a tiny forward pass through the first block
seq_len = 8
# This line creates a tensor of random integers, shaped like (1, seq_len).
# 0 - Lower bound (inclusive)
# gpt2.config.vocab_size - Upper bound (exclusive) — the number of tokens in GPT-2’s vocabulary (≈50,257 for GPT-2)
# (1, seq_len) - The shape of the tensor → one “batch” with 8 token IDs
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len))
print(dummy_tokens)
# Don’t track gradients for the operations inside this block.
# Normally, PyTorch runs in autograd mode, meaning:
#	•	Every operation on tensors is recorded in a computation graph.
#	•	This graph is used later to compute gradients (via backpropagation) during training.
# But in inference or evaluation, you don’t need gradients — you just want to run the model forward and get predictions.
with torch.no_grad():
    # Embed tokens (word embeddings)
    # this is a core line of how GPT-2 converts discrete tokens (IDs) into continuous vectors before feeding them into the transformer
    tok_emb = gpt2.transformer.wte(dummy_tokens)

    # Embed positions (positional encodings)
    # it gives the model a sense of token order (position).
    pos_emb = gpt2.transformer.wpe(torch.arange(seq_len).unsqueeze(0))

    # Combine token + positional embeddings
    # If tok_emb tells GPT-2 what each word is,
    # then pos_emb tells GPT-2 where each word is in the sentence.
    hidden_states = tok_emb + pos_emb

    # Forward through the first transformer block (Block 0)
    result = gpt2.transformer.h[0](hidden_states)

    # Optionally normalize (as GPT-2 does at the end of the stack)
    # Handle both return types (tuple or tensor)
    if isinstance(result, tuple):
        hidden_states = result[0]
    else:
        hidden_states = result

    print("Output shape after first block:", hidden_states.shape)

#print("\nOutput shape :", out.shape) # (batch, seq_len, hidden_size)

tensor([[48734, 18324, 35423, 24723, 15174,  5873, 15411, 12109]])
Output shape after first block: torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [35]:
# Print the name and modules inside gpt2
# Print only top-level modules
print("\n=== Top-level GPT-2 modules ===")
for name, module in gpt2.named_children():
    print(f"{name}: {type(module).__name__}")

# Print all nested modules recursively
print("\n=== All modules (recursively) ===")
for name, module in gpt2.named_modules():
    print(name, "→", module.__class__.__name__)


=== Top-level GPT-2 modules ===
transformer: GPT2Model
lm_head: Linear

=== All modules (recursively) ===
 → GPT2LMHeadModel
transformer → GPT2Model
transformer.wte → Embedding
transformer.wpe → Embedding
transformer.drop → Dropout
transformer.h → ModuleList
transformer.h.0 → GPT2Block
transformer.h.0.ln_1 → LayerNorm
transformer.h.0.attn → GPT2Attention
transformer.h.0.attn.c_attn → Conv1D
transformer.h.0.attn.c_proj → Conv1D
transformer.h.0.attn.attn_dropout → Dropout
transformer.h.0.attn.resid_dropout → Dropout
transformer.h.0.ln_2 → LayerNorm
transformer.h.0.mlp → GPT2MLP
transformer.h.0.mlp.c_fc → Conv1D
transformer.h.0.mlp.c_proj → Conv1D
transformer.h.0.mlp.act → NewGELUActivation
transformer.h.0.mlp.dropout → Dropout
transformer.h.1 → GPT2Block
transformer.h.1.ln_1 → LayerNorm
transformer.h.1.attn → GPT2Attention
transformer.h.1.attn.c_attn → Conv1D
transformer.h.1.attn.c_proj → Conv1D
transformer.h.1.attn.attn_dropout → Dropout
transformer.h.1.attn.resid_dropout → Dropout
tra

As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [55]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

model_name = "gpt2"   # this is the 124M model

# Load gpt2 model and tokenizer
gpt2 = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize input text
text = "Hello my name"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
ids = tokenizer.encode(text, return_tensors="pt")

# Get logits by passing the ids to the gpt2 model.
device = "cuda" if torch.cuda.is_available() else "cpu"
gpt2 = gpt2.to(device)
ids = ids.to(device)

with torch.no_grad():
  output = gpt2(ids)
  logits = output.logits
print("Logits shape :", logits.shape)

# Predict next token
# Extract logits for the *last token* in the sequence
last_token_logits = logits[0, -1, :]   # shape: (vocab_size,)

# Get the ID of the token with the highest logit (most likely next token)
predicted_token_id = torch.argmax(last_token_logits)

# Decode it back to a readable word
predicted_token = tokenizer.decode(predicted_token_id)

print(f"Next token predicted: '{predicted_token}'")

print("\nTop-5 predictions for the next token:")
"""
YOUR CODE HERE
"""


Logits shape : torch.Size([1, 3, 50257])
Next token predicted: ' is'

Top-5 predictions for the next token:


'\nYOUR CODE HERE\n'

### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
"""
YOUR CODE HERE
"""

def generate(model_key, prompt, strategy="greedy", max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    # Return the generations based on the provided strategy: greedy, top_k, top_p
    """
    YOUR CODE HERE
    """


In [ ]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))



Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [ ]:

tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top-p", 40))


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
tokenizers, models = {}, {}
# Load models and tokenizers
"""
YOUR CODE HERE
"""



We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [ ]:

tests=[("Once upon a time","greedy"),("What is 2+2?","top_k"),("Suggest a party theme.","top_p")]
for prompt,strategy in tests:
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key,prompt,strategy,80))


# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100):
    # Generation code
    """
    YOUR CODE HERE
    """

# Your code to build boxes, dropdowns, and other elements in the UI using widgets and creating the UI using widgets.vbox and display.
# Refer to https://ipywidgets.readthedocs.io/en/stable/
"""
YOUR CODE HERE
"""



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
